In [44]:
import warnings
warnings.simplefilter('ignore')
 
# будем отображать графики прямо в jupyter
%matplotlib inline
import matplotlib.pyplot as plt
#графики в svg выглядят более четкими
%config InlineBackend.figure_format = 'svg' 

#увеличим дефолтный размер графиков
from pylab import rcParams
rcParams['figure.figsize'] = 8, 5
import pandas as pd
from sklearn.tree import DecisionTreeClassifier       #Подключаем модуль с реализацие дерева решений
from sklearn.neighbors import KNeighborsClassifier    #Подключаем модуль с реализацией алгоритма ближайших соседей df_y = df['Churn']
df_X = df.drop(['Churn', 'State'], axis = 1)

Считаем датасет и посмотрим на первые 5 записей 

In [45]:
df = pd.read_csv('../../data/telecom_churn.csv')
df.head()

,State,Account length,Area code,International plan,Voice mail plan,Number vmail messages,Total day minutes,Total day calls,Total day charge,Total eve minutes,Total eve calls,Total eve charge,Total night minutes,Total night calls,Total night charge,Total intl minutes,Total intl calls,Total intl charge,Customer service calls,Churn
0,KS,128,415,No,Yes,25,265.1,110,45.07,197.4,99,16.78,244.7,91,11.01,10.0,3,2.70,1,False
1,OH,107,415,No,Yes,26,161.6,123,27.47,195.5,103,16.62,254.4,103,11.45,13.7,3,3.70,1,False
2,NJ,137,415,No,No,0,243.4,114,41.38,121.2,110,10.30,162.6,104,7.32,12.2,5,3.29,0,False
3,OH,84,408,Yes,No,0,299.4,71,50.90,61.9,88,5.26,196.9,89,8.86,6.6,7,1.78,2,False
4,OK,75,415,Yes,No,0,166.7,113,28.34,148.3,122,12.61,186.9,121,8.41,10.1,3,2.73,3,False


#### Разделим нашу таблицу на матрицу параметров df_x и целевой вектор df_y

In [46]:
df_y = df['Churn']
df_X = df.drop(['Churn', 'State'], axis = 1)

#### Заменим значения бинарных признаков International plan и Voice mail plan с Yes/No на 1/0

In [47]:
def swap(x):
    if x=='Yes':
        return 1
    else:
        return 0
df_X['International plan'] = df_X['International plan'].apply(swap)
df_X['Voice mail plan'] = df_X['Voice mail plan'].apply(swap)

#### Разделим нашу экспертную выборку на обучающую и тестовую

In [48]:
from sklearn.model_selection import train_test_split, StratifiedKFold
X_train, X_holdout, y_train, y_holdout = train_test_split(df_X.values, df_y.values, test_size = 0.3, random_state =17)

#### Используем дерево решений и knn для сравнения эффективности 

In [49]:
tree_clf = DecisionTreeClassifier(max_depth = 5, random_state = 17)
knn_clf = KNeighborsClassifier(n_neighbors = 10)

In [50]:
%%time
#скармливаем учебную выборку дереву решений 
tree_clf.fit(X_train, y_train)

CPU times: user 13.3 ms, sys: 2.29 ms, total: 15.6 ms
Wall time: 13.5 ms


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=5,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=17, splitter='best')

#### Отобразим на диаграмме получившееся дерево 

In [51]:
from sklearn.tree import export_graphviz
feature = df_X.columns

export_graphviz(tree_clf, feature_names =list(map(lambda x: str(x), feature)), out_file='pf_small_tree.dot', filled=True)
!dot -Tpng 'pf_small_tree.dot' -o 'pf_small_tree.png'

In [52]:
%%time
#скармливаем учебную выборку knn 
knn_clf.fit(X_train, y_train)

CPU times: user 3.57 ms, sys: 2.3 ms, total: 5.87 ms
Wall time: 4.07 ms


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=10, p=2,
                     weights='uniform')

#### используем самую простую метрику для оценки эффективности: доля правильных ответов в тестовой выборке

In [35]:
from sklearn.metrics import accuracy_score
tree_pred = tree_clf.predict(X_holdout)
accuracy_score(y_holdout, tree_pred)

0.94

In [36]:
knn_pred = knn_clf.predict(X_holdout)
accuracy_score(y_holdout, knn_pred)

0.881

#### Теперь настроим параметры дерева на кросс-валидации

In [37]:
from sklearn.model_selection import GridSearchCV, cross_val_score

tree_params={'max_depth': range(1, 11),
             'max_features': range(4, 19)}

In [38]:
#инициализируем дерево решений с кросс-валидацией и скармливаем ему учебную выборку 
tree_grid = GridSearchCV(tree_clf, tree_params, cv=5, n_jobs=-1, verbose =True)
tree_grid.fit(X_train, y_train)

Fitting 5 folds for each of 150 candidates, totalling 750 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  68 tasks      | elapsed:    1.6s
[Parallel(n_jobs=-1)]: Done 750 out of 750 | elapsed:    3.6s finished


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=DecisionTreeClassifier(class_weight=None,
                                              criterion='gini', max_depth=5,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort=False, random_state=17,
                                              splitter='best'),
             iid='warn', n_jobs=-1,
             param_grid={'max_depth': range(1, 11),
                         'max_features': range(4, 19)},
             pre_dispatch='2*n_jobs

In [14]:
#смотрим лучшие параметры дерева
tree_grid.best_params_

{'max_depth': 6, 'max_features': 17}

In [15]:
#смотрим прогнозируемость на этих лучших параметрах на k-ой части учебной выборки отложенной на этапе кроссвалидации
tree_grid.best_score_

0.9425632233176168

In [16]:
#смотрим прогнозируемость на этих лучших параметрах на тестовой выборке
accuracy_score(y_holdout, tree_grid.predict(X_holdout))

0.948

#### Настроим кроссвалидацию на knn

In [17]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

knn_pipe = Pipeline([('scaler', StandardScaler()), ('knn',KNeighborsClassifier(n_jobs=-1))])
knn_params = {'knn__n_neighbors': range(1, 10)}

In [18]:
knn_grid = GridSearchCV(knn_pipe, knn_params, cv=5, n_jobs=-1, verbose = True)

In [19]:
knn_grid.fit(X_train, y_train)

Fitting 5 folds for each of 9 candidates, totalling 45 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  45 out of  45 | elapsed:    3.0s finished


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('scaler',
                                        StandardScaler(copy=True,
                                                       with_mean=True,
                                                       with_std=True)),
                                       ('knn',
                                        KNeighborsClassifier(algorithm='auto',
                                                             leaf_size=30,
                                                             metric='minkowski',
                                                             metric_params=None,
                                                             n_jobs=-1,
                                                             n_neighbors=5, p=2,
                                                             weights='uniform'))],
                                verbose=False

In [20]:
#смотрим прогнозируемость на этих лучших параметрах на k-ой части учебной выборки
knn_grid.best_params_, knn_grid.best_score_

({'knn__n_neighbors': 7}, 0.8859837119588513)

In [22]:
#смотрим прогнозируемость на этих лучших параметрах на тестовой выборке
accuracy_score(y_holdout, knn_grid.predict(X_holdout))

0.89